# 🎥 YouTube Livestream Summarizer

AI-Powered Real-Time Livestream Analysis using FFmpeg and Google's Gemini AI.

**GitHub Repository**: [youtube-livestream-summarizer](https://github.com/leoparker157/youtube-livestream-summarizer)

---

## 📦 Step 1: Install Dependencies

This cell installs all required packages including FFmpeg, yt-dlp, and Python libraries.

In [ ]:
%%capture
# Install system dependencies
!apt-get update -qq
!apt-get install -y ffmpeg > /dev/null 2>&1

# Install Python packages
!pip install -q gradio google-generativeai python-dotenv schedule yt-dlp

print("✅ All dependencies installed successfully!")

## 🔑 Step 2: Set Your Gemini API Key

Get your free API key from [Google AI Studio](https://aistudio.google.com/app/apikey)

**Note**: You can also enter it directly in the Gradio interface.

In [ ]:
import os

# Option 1: Set API key here (recommended for Colab)
GEMINI_API_KEY = ""  # Paste your API key here

# Option 2: Use Colab secrets (more secure)
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    if GEMINI_API_KEY:
        print("✅ API key set manually")
    else:
        print("⚠️ No API key set. You can enter it in the Gradio interface.")

os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

## 📥 Step 3: Clone Repository and Load Application

This downloads the latest version of the summarizer from GitHub.

In [ ]:
# Clone the repository
!git clone https://github.com/leoparker157/youtube-livestream-summarizer.git
%cd youtube-livestream-summarizer

print("✅ Repository cloned successfully!")

## ✅ Step 3.5: Verify Installation

Check that all dependencies are properly installed.

In [ ]:
# Verify setup
import subprocess
import os

print("🔍 Verifying setup...")
print("="*50)

# Check FFmpeg
try:
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ FFmpeg: Installed")
    else:
        print("❌ FFmpeg: Not found")
except:
    print("❌ FFmpeg: Not found")

# Check yt-dlp
try:
    result = subprocess.run(['yt-dlp', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ yt-dlp: Installed (version {result.stdout.strip()})")
    else:
        print("❌ yt-dlp: Not found")
except:
    print("❌ yt-dlp: Not found")

# Check Python packages
packages = ['gradio', 'google.generativeai', 'dotenv']
for package in packages:
    try:
        __import__(package)
        print(f"✅ {package}: Installed")
    except ImportError:
        print(f"❌ {package}: Not installed")

# Check files
if os.path.exists('gradio_app.py'):
    print("✅ gradio_app.py: Found")
else:
    print("❌ gradio_app.py: Not found")

print("="*50)
print("✅ Verification complete!")

## 🚀 Step 4: Launch Gradio Interface

**Note**: The public URL may take 30-60 seconds to generate. Wait for the link to appear.

In [ ]:
# Import and run the Gradio app
import sys
import os
sys.path.append('.')

# Create segments directory if it doesn't exist
os.makedirs('segments', exist_ok=True)

print("📂 Loading application...")

try:
    from gradio_app import create_interface
    
    print("✅ Application loaded successfully!")
    print("🚀 Launching Gradio interface...")
    print("⏳ This may take 30-60 seconds to generate the public URL...")
    
    # Create and launch the interface
    demo = create_interface()
    demo.launch(share=True, debug=False)  # Set debug=False to avoid blocking
except Exception as e:
    print(f"❌ Error loading application: {e}")
    print("\n🔍 Troubleshooting:")
    print("1. Make sure all dependencies are installed")
    print("2. Check that the repository was cloned correctly")
    print("3. Verify FFmpeg is installed")
    import traceback
    traceback.print_exc()

---

In [ ]:
# Quick diagnostic test
import sys
import os

print("🔍 Quick Diagnostic Test")
print("="*50)
print(f"Current directory: {os.getcwd()}")
print(f"Python path: {sys.path[:3]}")
print(f"Files in current dir: {os.listdir('.')[:10]}")

# Try importing the app
try:
    from gradio_app import LivestreamSummarizerGradio
    print("✅ Successfully imported LivestreamSummarizerGradio")
    
    # Try creating instance
    summarizer = LivestreamSummarizerGradio()
    print("✅ Successfully created summarizer instance")
    print("✅ App should work! Try running Step 4 again.")
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Fix: Make sure you ran Step 3 to clone the repository")

## 🔧 Troubleshooting

### If the Gradio interface won't load:

1. **Check the verification step above** - Make sure all dependencies are installed
2. **Restart the runtime** - Runtime → Restart runtime, then run all cells again
3. **Check console output** - Look for error messages in the cell output
4. **Try manual launch** - Run this test cell below to see detailed errors

### Common Issues:

- **"Module not found"**: Re-run Step 1 to reinstall dependencies
- **"FFmpeg not found"**: Re-run the apt-get install command
- **URL keeps loading**: Wait 60 seconds, the Gradio tunnel takes time
- **"No module named gradio_app"**: Make sure you're in the youtube-livestream-summarizer directory

## 🧹 Optional: Cleanup

Run this cell to clean up files after you're done.

In [ ]:
# Clean up temporary files
!rm -rf segments/
!rm -f compressed.mp4
!rm -f *.txt

print("✅ Cleanup complete!")